In [1]:
import pandas as pd
import os
from datetime import datetime
import warnings
import sys
sys.path.append('../src')
from functions import stock_selection_weight_allocation, adjust_portfolio, generate_and_save_data
warnings.filterwarnings("ignore")

In [2]:
all_stocks_df = pd.read_csv('../data/all_stock_data.csv', index_col=0)
all_stocks_df.index = pd.to_datetime(all_stocks_df.index)
all_stocks_df = all_stocks_df.sort_index()

In [3]:
# start_date = '2010-01-01'
# end_date = str(datetime.today())[:10]

# generate_and_save_data(start_date, end_date)

In [4]:
final_symbols = list(all_stocks_df.columns)

In [5]:
len(final_symbols)

751

In [6]:
# Generating a list of the first day of every month from Jan 2020 to Dec 2023
monthly_dates = [datetime(year, month, 1) for year in range(2020, 2024) for month in range(1, 13)]
formatted_monthly_dates = [date.strftime('%Y-%m-%d') for date in monthly_dates]

# Generating a list of the first day of every quarter from Jan 2020 to Dec 2023
quarterly_dates = [datetime(year, month, 1) for year in range(2020, 2024) for month in [1, 4, 7, 10]]
formatted_quarterly_dates = [date.strftime('%Y-%m-%d') for date in quarterly_dates]

backtesting_buying_dates_for_1m_hold = formatted_monthly_dates
backtesting_buying_dates_for_1q_hold = formatted_quarterly_dates

In [7]:
backtesting_buying_dates_for_1m_hold, backtesting_buying_dates_for_1q_hold

(['2020-01-01',
  '2020-02-01',
  '2020-03-01',
  '2020-04-01',
  '2020-05-01',
  '2020-06-01',
  '2020-07-01',
  '2020-08-01',
  '2020-09-01',
  '2020-10-01',
  '2020-11-01',
  '2020-12-01',
  '2021-01-01',
  '2021-02-01',
  '2021-03-01',
  '2021-04-01',
  '2021-05-01',
  '2021-06-01',
  '2021-07-01',
  '2021-08-01',
  '2021-09-01',
  '2021-10-01',
  '2021-11-01',
  '2021-12-01',
  '2022-01-01',
  '2022-02-01',
  '2022-03-01',
  '2022-04-01',
  '2022-05-01',
  '2022-06-01',
  '2022-07-01',
  '2022-08-01',
  '2022-09-01',
  '2022-10-01',
  '2022-11-01',
  '2022-12-01',
  '2023-01-01',
  '2023-02-01',
  '2023-03-01',
  '2023-04-01',
  '2023-05-01',
  '2023-06-01',
  '2023-07-01',
  '2023-08-01',
  '2023-09-01',
  '2023-10-01',
  '2023-11-01',
  '2023-12-01'],
 ['2020-01-01',
  '2020-04-01',
  '2020-07-01',
  '2020-10-01',
  '2021-01-01',
  '2021-04-01',
  '2021-07-01',
  '2021-10-01',
  '2022-01-01',
  '2022-04-01',
  '2022-07-01',
  '2022-10-01',
  '2023-01-01',
  '2023-04-01',
  '2023

In [8]:
len(backtesting_buying_dates_for_1m_hold), len(backtesting_buying_dates_for_1q_hold)

(48, 16)

In [9]:
govt_bond_df = pd.read_csv('../data/India 10-Year Bond Yield Historical Data.csv')
govt_bond_df = govt_bond_df[['Date','Price']]
govt_bond_df.index = govt_bond_df['Date']
govt_bond_df = govt_bond_df.drop('Date', axis=1)
govt_bond_df.index = pd.to_datetime(govt_bond_df.index)

In [10]:
# Function to simulate the portfolio over the period
def backtest_portfolio(initial_investment, stock_selection_strategy, weight_allocation_strategy, rebalance_dates, backtesting_file_name, last_x_years, last_x_years_opt):

    filters = 4

    if stock_selection_strategy in [1,2,5,6,9,10,13,14]:
        holding_period = '1q'
    elif stock_selection_strategy in [3,4,7,8,11,12,15,16]:
        holding_period = '1m'

    if stock_selection_strategy in [1,3,5,7,9,11,13,15]:
        returns_type = 'SR'
    elif stock_selection_strategy in [2,4,6,8,10,12,14,16]:
        returns_type = 'LR'

    if stock_selection_strategy in [1,2,3,4]:
        max_non_positive_returns_count = 15
    elif stock_selection_strategy in [5,6,7,8]:
        max_non_positive_returns_count = 10
    elif stock_selection_strategy in [9,10,11,12]:
        max_non_positive_returns_count = None
        filters = 3
    elif stock_selection_strategy in [13,14,15,16]:
        max_non_positive_returns_count = None
        filters = 2

    current_cash = initial_investment
    transaction_records = []

    for i in range(len(rebalance_dates)):

        buy_date = rebalance_dates[i]

        print(buy_date, current_cash)

        # Get the portfolio based on current strategy
        portfolio, sell_date, best_method = stock_selection_weight_allocation(buy_date, holding_period, returns_type, max_non_positive_returns_count, weight_allocation_strategy, all_stocks_df, govt_bond_df, filters, last_x_years, last_x_years_opt)
        
        portfolio = adjust_portfolio(portfolio)
        
        if portfolio != None:

            # Calculate the allocated cash for each stock in the portfolio
            allocated_cash = {stock: current_cash * weight for stock, weight in portfolio.items()}

            # Adjust buy_date and sell_date to the nearest next available date in all_stocks_df
            adjusted_buy_date = all_stocks_df[all_stocks_df.index >= buy_date].index[0]
            try:
                adjusted_sell_date = all_stocks_df[all_stocks_df.index >= sell_date].index[0]
            except:
                adjusted_sell_date = all_stocks_df[all_stocks_df.index <= sell_date].index[-1]

            # Get stock prices on buying and selling dates
            buy_prices = all_stocks_df.loc[adjusted_buy_date, portfolio.keys()]
            sell_prices = all_stocks_df.loc[adjusted_sell_date, portfolio.keys()]

            # Calculate the number of shares bought for each stock
            buy_quantities = {stock: allocated_cash[stock] / buy_prices[stock] for stock in portfolio.keys()}

            # Calculate the value of the portfolio at selling date
            sell_values = {stock: sell_prices[stock] * buy_quantities[stock] for stock in portfolio.keys()}
            
            current_cash = round(sum(sell_values.values()),2)
        
        # Record the transaction details
        transaction = {
            'buy_date': adjusted_buy_date,
            'sell_date': adjusted_sell_date,
            'portfolio': portfolio,
            'investment_value': current_cash
        }
        
        print(len(portfolio), portfolio)

        transaction_records.append(transaction)

    # Convert transaction records to DataFrame
    df = pd.DataFrame(transaction_records)
    # Save to CSV
    file_name = f"../backtesting_results/version_{backtesting_file_name}/strategy_{stock_selection_strategy}_{weight_allocation_strategy}.csv"
    df.to_csv(file_name, index=False)

    return current_cash

In [11]:
# Initial investment amount
initial_investment = 100000.00  # Example amount

# Dictionary to store final values of each strategy combination
strategy_performance = {}

last_x_years_last_x_years_opt = 1

# Iterate over all strategy combinations
for last_x_years in [1]:
    for last_x_years_opt in [1]:
        for stock_selection_strategy in range(1, 17):
            for weight_allocation_strategy in range(1, 12):

                file_name = f"../backtesting_results/version_{last_x_years_last_x_years_opt}/strategy_{stock_selection_strategy}_{weight_allocation_strategy}.csv"

                # Check if the file for this combination already exists
                if os.path.exists(file_name):
                    print(f"Skipping simulation for Stock Selection Strategy {stock_selection_strategy} and Weight Allocation Strategy {weight_allocation_strategy} as file already exists.\n")
                    continue

                if stock_selection_strategy in [1,2,5,6,9,10,13,14]:
                    holding_period = '1q'
                elif stock_selection_strategy in [3,4,7,8,11,12,15,16]:
                    holding_period = '1m'

                # Determine rebalancing schedule
                rebalance_dates = backtesting_buying_dates_for_1m_hold if holding_period == '1m' else backtesting_buying_dates_for_1q_hold

                print(f"Simulation for Stock Selection Strategy {stock_selection_strategy} and Weight Allocation Strategy {weight_allocation_strategy} started.")

                # Backtest and store the final portfolio value
                final_value = backtest_portfolio(initial_investment, stock_selection_strategy, weight_allocation_strategy, rebalance_dates, last_x_years_last_x_years_opt, last_x_years, last_x_years_opt)
                print("\n", stock_selection_strategy, weight_allocation_strategy, final_value, "\n")

        last_x_years_last_x_years_opt += 12

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 1 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 2 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 3 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 4 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 5 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 6 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 7 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 8 as file already exists.

Skipping simulation for Stock Selection Strategy 1 and Weight Allocation Strategy 9 as file already exists.

Skipping simulation